# Exploratory data analysis of food images 

This notebook will do the exploratory data analysis of the food images.

### EDA checklist

* What question(s) are you trying to solve (or prove wrong)?
* What kind of data do you have and how do you treat different types?
* What’s missing from the data and how do you deal with it?
* Where are the outliers and why should you care about them?
* How can you add, change or remove features to get more out of your data?

The bigger goal is to <span style="color:blue">recognize food from images</span>. Starting with analysing the amount and kind of data present for this project.